In [53]:
!echo 'Welcome to simple Souffle graph benchmarking.'
!echo 'Behold the power of The Machine:'
!echo "CPUs: $(nproc) / RAM: $(free -h | awk '/^Mem:/ {print $2}')"

Welcome to simple Souffle graph benchmarking.
Behold the power of The Machine:
CPUs: 32 / RAM: 125Gi


In [54]:
!pwd

/home/evgenys/graph_bench


In [55]:
!ls

benchmark_logica.ipynb	 sg_for_tree10.s      sg_tree11.csv  tc_g3k.csv
benchmark_souffle.ipynb  sg_for_tree11.csv.s  sg_tree12.csv  tc_g4k.csv
g1k.csv			 sg_for_tree11.s      sg_tree7.csv   tc_g5k.csv
g2k.csv			 sg_for_tree12.csv.s  sg_tree8.csv   tree10.csv
g3k.csv			 sg_for_tree12.s      sg_tree9.csv   tree11.csv
g4k.csv			 sg_for_tree7.csv.s   tc_for_g1k.s   tree12.csv
g5c.csv			 sg_for_tree7.s       tc_for_g2k.s   tree5.csv
g5k.csv			 sg_for_tree8.csv.s   tc_for_g3k.s   tree7.csv
graph_bench		 sg_for_tree8.s       tc_for_g4k.s   tree8.csv
graphs.db		 sg_for_tree9.csv.s   tc_for_g5k.s   tree9.csv
logica			 sg_for_tree9.s       tc_g1k.csv
sg_for_tree10.csv.s	 sg_tree10.csv	      tc_g2k.csv


In [56]:
from IPython.core.magic import register_cell_magic
from IPython import get_ipython
import time
from logica.common import sqlite3_logica
import pandas

timing = {}

reports = []

@register_cell_magic
def loop(line, cell):
    global timing
    local_timing = {}
    ip = get_ipython()
    # Evaluate the line to get the list (e.g., "my_files")
    problem_name, should_time, iterator = ip.ev(line) 
    
    for item in iterator:
        # Inject 'item' into global namespace so the inner magic sees it
        ip.user_ns['loop_parameter'] = item 
        # Run the content as a new cell execution
        start_time = time.perf_counter()
        ip.run_cell(cell.replace('{loop_parameter}', item))
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        if should_time:
          timing[item] = elapsed
          local_timing[item] = elapsed
    if should_time:
      report = (' === Timing for %s ===\n' % problem_name) + (
          sqlite3_logica.DataframeAsArtisticTable(pandas.DataFrame({'problem': list(local_timing.keys()),
                                                                  'time': list(local_timing.values())})))
      reports.append(report)
      print(report)
    


In [57]:
graphs = ['g1k', 'g2k', 'g3k', 'g4k', 'g5k']

In [58]:
%%loop ('writing TC programs', False, graphs)
%%writefile tc_for_{loop_parameter}.s

.decl g(a:number,b:number)
.decl tc(a:number, b:number)
.decl result(a:number, b:number)

.input g(filename="{loop_parameter}.csv",delimiter=",",headers=true)

tc(a, b) :- g(a, b).
tc(a, c) :- tc(a, b), g(b, c).

result(a, b) :- tc(a, b).

.output result(filename="tc_{loop_parameter}.csv",delimiter=",")

Overwriting tc_for_g1k.s
Overwriting tc_for_g2k.s
Overwriting tc_for_g3k.s
Overwriting tc_for_g4k.s
Overwriting tc_for_g5k.s


In [59]:
!ls

benchmark_logica.ipynb	 sg_for_tree10.s      sg_tree11.csv  tc_g3k.csv
benchmark_souffle.ipynb  sg_for_tree11.csv.s  sg_tree12.csv  tc_g4k.csv
g1k.csv			 sg_for_tree11.s      sg_tree7.csv   tc_g5k.csv
g2k.csv			 sg_for_tree12.csv.s  sg_tree8.csv   tree10.csv
g3k.csv			 sg_for_tree12.s      sg_tree9.csv   tree11.csv
g4k.csv			 sg_for_tree7.csv.s   tc_for_g1k.s   tree12.csv
g5c.csv			 sg_for_tree7.s       tc_for_g2k.s   tree5.csv
g5k.csv			 sg_for_tree8.csv.s   tc_for_g3k.s   tree7.csv
graph_bench		 sg_for_tree8.s       tc_for_g4k.s   tree8.csv
graphs.db		 sg_for_tree9.csv.s   tc_for_g5k.s   tree9.csv
logica			 sg_for_tree9.s       tc_g1k.csv
sg_for_tree10.csv.s	 sg_tree10.csv	      tc_g2k.csv


In [60]:
!cat tc_for_g2k.s


.decl g(a:number,b:number)
.decl tc(a:number, b:number)
.decl result(a:number, b:number)

.input g(filename="g2k.csv",delimiter=",",headers=true)

tc(a, b) :- g(a, b).
tc(a, c) :- tc(a, b), g(b, c).

result(a, b) :- tc(a, b).

.output result(filename="tc_g2k.csv",delimiter=",")


In [61]:
%%loop ('Souffle TC', True, graphs)
!echo 'Running TC for {loop_parameter}'
!souffle tc_for_{loop_parameter}.s --jobs auto

Running TC for g1k
Running TC for g2k
Running TC for g3k
Running TC for g4k
Running TC for g5k
 === Timing for Souffle TC ===
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| g1k     | 0.6723721469998054 |
| g2k     | 1.9730765969998174 |
| g3k     | 4.463873752999916  |
| g4k     | 8.292752597999879  |
| g5k     | 13.028028106999955 |
+---------+--------------------+


In [62]:
trees = ['tree7', 'tree8', 'tree9', 'tree10', 'tree11', 'tree12']

In [63]:
%%loop ('writing SG programs', False, trees)
%%writefile sg_for_{loop_parameter}.s

.decl g(a:number,b:number)
.decl sg(a:number, b:number)
.decl result(a:number, b:number)

.input g(filename="{loop_parameter}.csv",delimiter=",",headers=true)

sg(a, b) :- g(x, a), g(x, b).
sg(a, b) :- sg(x, y), g(x, a), g(y, b).

result(a, b) :- sg(a, b).

.output result(filename="sg_{loop_parameter}.csv",delimiter=",")

Overwriting sg_for_tree7.s
Overwriting sg_for_tree8.s
Overwriting sg_for_tree9.s
Overwriting sg_for_tree10.s
Overwriting sg_for_tree11.s
Overwriting sg_for_tree12.s


In [64]:
%%loop ('Souffle SG', True, trees)
!echo 'Running SG for {loop_parameter}'
!souffle sg_for_{loop_parameter}.s --jobs auto

Running SG for tree7
Running SG for tree8
Running SG for tree9
Running SG for tree10
Running SG for tree11
Running SG for tree12
 === Timing for Souffle SG ===
+---------+---------------------+
| problem | time                |
+---------+---------------------+
| tree7   | 0.2391701319997992  |
| tree8   | 0.29926064700021016 |
| tree9   | 0.6816575349998857  |
| tree10  | 4.36850722500003    |
| tree11  | 29.756299892000243  |
| tree12  | 175.14233424400027  |
+---------+---------------------+


In [65]:
for report in reports:
    print(report)

 === Timing for Souffle TC ===
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| g1k     | 0.6723721469998054 |
| g2k     | 1.9730765969998174 |
| g3k     | 4.463873752999916  |
| g4k     | 8.292752597999879  |
| g5k     | 13.028028106999955 |
+---------+--------------------+
 === Timing for Souffle SG ===
+---------+---------------------+
| problem | time                |
+---------+---------------------+
| tree7   | 0.2391701319997992  |
| tree8   | 0.29926064700021016 |
| tree9   | 0.6816575349998857  |
| tree10  | 4.36850722500003    |
| tree11  | 29.756299892000243  |
| tree12  | 175.14233424400027  |
+---------+---------------------+
